In [90]:
import pandas as pd
import numpy as np


In [91]:
aapl=pd.read_csv('/Users/apple/Desktop/Python-Finance-QuantConnect/DATA/AAPL.csv', index_col='Date', parse_dates=True)

In [92]:
fb=pd.read_csv('/Users/apple/Desktop/Python-Finance-QuantConnect/DATA/FB.csv', index_col='Date', parse_dates=True)

In [93]:
fb.info()
aapl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2016-09-06 to 2021-09-02
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1258 non-null   float64
 1   High       1258 non-null   float64
 2   Low        1258 non-null   float64
 3   Close      1258 non-null   float64
 4   Adj Close  1258 non-null   float64
 5   Volume     1258 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 68.8 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2016-09-06 to 2021-09-02
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1258 non-null   float64
 1   High       1258 non-null   float64
 2   Low        1258 non-null   float64
 3   Close      1258 non-null   float64
 4   Adj Close  1258 non-null   float64
 5   Volume     1258 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 68.8 KB


In [94]:
aapl['DailyRet']=aapl['Adj Close'].pct_change(1)
fb['DailyRet']=fb['Adj Close'].pct_change(1)


In [95]:
fb=fb.dropna()
aapl=aapl.dropna()

In [96]:
fb
aapl

,Open,High,Low,Close,Adj Close,Volume,DailyRet
Date,,,,,,,
2016-09-07,26.957500,27.190001,26.767500,27.090000,25.406099,169457200,0.006128
2016-09-08,26.812500,26.817499,26.309999,26.379999,24.740232,212008000,-0.026209
2016-09-09,26.160000,26.430000,25.782499,25.782499,24.179869,186228000,-0.022650
2016-09-12,25.662500,26.430000,25.632500,26.360001,24.721476,181171200,0.022399
2016-09-13,26.877501,27.197500,26.809999,26.987499,25.309967,248704800,0.023805
...,...,...,...,...,...,...,...
2021-08-27,147.479996,148.750000,146.830002,148.600006,148.600006,55721500,0.007185
2021-08-30,149.000000,153.490005,148.610001,153.119995,153.119995,90956700,0.030417
2021-08-31,152.660004,152.800003,151.289993,151.830002,151.830002,86453100,-0.008425


In [97]:
aapl['DailyRet'].std()

0.019046635958637193

In [135]:
def compSR(df, rf=0):
    meanRet=df['DailyRet'].mean()
    stdDev=df['DailyRet'].std()
    SR=((meanRet-rf)/stdDev)
    return SR

In [136]:
aaplSR=compSR(aapl)

In [137]:
aaplSR

0.08500663732501317

In [138]:
fbSR=compSR(fb)

In [139]:
fbSR

0.051462535982218956

In [140]:
def Sortino(df, threshold=0,rf=0.0):
    meanRet=df['DailyRet'].mean()
    downside_risk=df[df['DailyRet']<threshold]['DailyRet']
    stdDev=downside_risk.std()
    SortinoRatio=(meanRet-rf)/stdDev
    return SortinoRatio* (252**0.5)

In [141]:
aaplSortino=Sortino(aapl)
aaplSortino

1.7714166312409234

In [142]:
fbSortino=Sortino(fb)
fbSortino

1.033767721917672

In [143]:
import scipy.stats

In [149]:
def PSR(df, benchmark=0):
    sr=compSR(df)
    skew=scipy.stats.skew(df['DailyRet'])
    kurtosis=scipy.stats.kurtosis(df['DailyRet'], fisher=True)
    n=len(df)
    sigmaSR=( (1/(n-1)) * ( 1+ 0.5 *sr**2  +skew*sr+ (kurtosis/4) *sr**2)  )**0.5
    finalr= (sr-benchmark)/sigmaSR
    psr=scipy.stats.norm.cdf(finalr)
    return psr

In [150]:
aaplPSR=PSR(aapl)

In [153]:
aaplPSR* (252**.5)

15.853081185964554

In [154]:
fbPSR=PSR(fb)
fbPSR* (252**.5)

15.360761358206695